# EVENT PLOTS -- GUI version

A NB template to produce event plots (GUI version)

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
%gui qt
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import time
import tables as tb
import numpy as np

In [ ]:
from invisible_cities.database import load_db

In [ ]:
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.sierpe.blr as blr
import invisible_cities.reco.peak_functions_c as cpf
from invisible_cities.core.mpl_functions import set_plot_labels
from invisible_cities.reco.params import S12Params, ThresholdParams
from   invisible_cities.core.system_of_units_c import units

In [ ]:
from icaro.core.gui_functions import fplot_pmt_waveforms, fplot_pmt_signals_vs_time_mus, fplot_signal_vs_time_mus,\
                           fplot_s12, hist_1d

In [ ]:
from icaro.core.event_pmaps import EventPmaps, print_s12, print_s2si

### Import GUI

In [ ]:
from qt_gui import QtGui

In [ ]:
gui=QtGui()
gui.show()

#### Function scan_events loops over events and plots waveforms

In [ ]:
def scan_events(epm, gui, pmtrwf, sipmrwf, 
                event_list=list(range(1)), 
                event_plot_list=list(range(1)), 
                printout=10):
    """Loop over events"""
    
    for event in event_list:
        if(event%printout == 0):
            print('++++++processing event number {}'.format(event))
        epm.calibrated_pmt_and_csum(event, pmtrwf)
        epm.calibrated_sipm(event, sipmrwf)
        
        if(event%printout == 0):
            print('--S1 search')
        epm.find_s1()
        
        if(event%printout == 0):
            print('--S2 search')
        epm.find_s2()
        
        if (len(epm.S2) > 0):
            epm.find_s2si()
        
        if event in event_plot_list:
            gui.add_figure('RWF-event-{}'.format(event), 
                            fplot_pmt_waveforms(epm.RWF, zoom=False, window_size=10000))
            gui.add_figure('CWF_vs_time_mus-event-{}'.format(event), 
                            fplot_pmt_signals_vs_time_mus(epm.CWF,
                                                         epm.P.pmt_active,
                                                         t_min      =    0,
                                                         t_max      = 1300,
                                                         signal_min =    -5,
                                                         signal_max =  200))
            gui.add_figure('Calibrated_SUM-event-{}'.format(event), 
                            fplot_signal_vs_time_mus(epm.csum, t_min=0, t_max=1300, signal_min=-5, signal_max=100))
            if (len(epm.S1) > 0):
                gui.add_figure('S1-event-{}'.format(event), fplot_s12(epm.S1))
            if (len(epm.S2) > 0):
                gui.add_figure('S2-event-{}'.format(event), fplot_s12(epm.S2))

## Define here the  events to analyze

### Krypton

1. Run 3389

Start Date     End Date     Duration (sec)

2017-03-23 20:52:16     2017-03-25 11:22:54     138638

trigger rate: 4.65 evt/s

events: 645.505

Trigger in 5 pmts with:

1. baseline dev: 15
2. max amplitude: 1000
3. qmin: 3000
4. qmax: 20000
5. min time: 4us
6. max time: 12us

#### PMAPS for Kr

#### Files for Kr

RWF_file_kr = os.path.join(os.environ['IC_DATA'],
                        'LSC/wvfms/{}/dst_waves.gdcsnext.000_{}.root.h5'.format(run_number, run_number))

### Na-22

1. Run 3574, 3575
2. Threshold baseline 20 counts
3. Qmin = 90,0000 counts
4. width min > 7 mus
5. 5 PMTs / 11

#### PMAPS parameters Na-22

#### Files parameters Na-22

### Alphas

1. run_number = 3452

#### PMAPS parameters for alphas

s1par  = S12Params(tmin=0*units.mus, tmax=650*units.mus, lmin=10, lmax=30, stride=4, rebin=False)
s2par = S12Params(tmin=630*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)
thr    = ThresholdParams(thr_s1=10 * units.pes,  thr_s2=10 *units.pes,
                        thr_MAU = 3 * units.adc, thr_sipm = 3.5 * units.pes,
                        thr_SIPM = 30 * units.adc)

#### Files for alphas

## Chose events to analyze

#### event type: alphas

In [ ]:
run_number = 3452

In [ ]:
DataPMT = load_db.DataPMT(run_number)

#### Files

In [ ]:
RWF_file = os.path.join(os.environ['IC_DATA'],
                        'LSC/wvfms/{}/dst_waves.gdcsnext.001_{}.root.h5'.format(run_number, run_number))




### Define selection parameters for PMAPS

In [ ]:
s1par  = S12Params(tmin=0*units.mus, tmax=650*units.mus, lmin=10, lmax=30, stride=4, rebin=False)
s2par = S12Params(tmin=630*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)
thr    = ThresholdParams(thr_s1=10 * units.pes,  thr_s2=10 *units.pes,
                        thr_MAU = 3 * units.adc, thr_sipm = 3.5 * units.pes,
                        thr_SIPM = 30 * units.adc)

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

In [ ]:
epm = EventPmaps(run_number, s1par, s2par, thr, verbose=True)

In [ ]:
n_events = 5
sample_plot = 1
t0 = time.time()
scan_events(epm, gui, pmtrwf, sipmrwf, 
            event_list=list(range(n_events)), 
            event_plot_list=list(range(0,n_events,sample_plot)),
           printout=1)
t1 = time.time()        
dt = t1 - t0
print("run {} events in {} s event/s = {}".format(n_events, dt, dt/n_events))